### UVR
[Link](https://totoro.banrep.gov.co/analytics/saw.dll?Go&Action=prompt&Path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20UPAC%20-%20UVR%2F1.1%20UVR%2F1.1.2.UVR_Serie%20historica%20diaria&Options=rdf&lang=es&NQUser=publico&NQPassword=publico123)

In [9]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from datetime import datetime
import selenium as sel
import pandas as pd
import time
import glob
import sys
import os

cwd = os.getcwd()
cwd = cwd.replace("Notebooks", "Scripts")
sys.path.insert(0, cwd.replace("\\\\", "\\"))
import base as bs

In [10]:
path = 'C:/Users/'+os.getlogin()+'/OneDrive - Accenture/ACNCustomerAnalysis/Indicadores_Macro'

In [39]:
def extraer_UVR(start_date: str, end_date: str):
    """Función que ingresa a la página del banco de la república y descarga los datos de UVR. 
    Almacena el arcivo en la carpeta de data cruda.

    Args:
        driver (sel.webdriver.Edge): driver de selenium
        conservar (bool, optional): True para dejar el rango por defecto, False para establecerlo. Predeterminado a True.
        fecha_inicio (str, optional): Fecha desde la que se quieren extraer los datos. Predeterminado a '01/01/2000'.
        fecha_final (str, optional): Fecha hasta la que se quieren extraer los datos. Predeterminado a datetime.today().strftime("%d/%m/%Y").
    """
    driver = bs.ejecutar_driver('https://totoro.banrep.gov.co/analytics/saw.dll?Go&Action=prompt&Path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20UPAC%20-%20UVR%2F1.1%20UVR%2F1.1.2.UVR_Serie%20historica%20diaria&Options=rdf&lang=es&NQUser=publico&NQPassword=publico123')
    inputs = driver.find_elements(By.XPATH, './/input')
    
    desde = inputs[0]
    hasta = inputs[1]
    aceptar = inputs[2]

    desde.clear()
    desde.send_keys(start_date.replace('-', '/'))
    
    hasta.clear()
    hasta.send_keys(end_date.replace('-', '/'))
        
    aceptar.click()
    
    time.sleep(20)
    
    driver.find_elements(By.XPATH, './/td[@class="ResultLinksCell"]')[2].click()
    aux = driver.find_elements(By.XPATH, './/td[@class="masterMenu shadowMenuCell"]')[2]
    aux.find_elements(By.XPATH, './/a')[4].click()
    driver.find_element(By.XPATH, './/a[@aria-label="CSV"]').click()
    
    time.sleep(10)
    driver.quit()
    
    fuente_archivo = bs.obtener_nombre_descarga('/Users/'+os.getlogin()+'/Downloads')
    
    df = pd.read_csv(fuente_archivo)
    df['date'] = df['Fecha (dd/mm/aaaa)'].str.split('-').map(lambda x: '-'.join(x[::-1]))
    
    try: 
        start = df[df['date'] == start_date].index[0]
    except IndexError:
        start = 0
        
    try: 
        end = df[df['date'] == end_date].index[0]
    except IndexError:
        end = len(df) - 1
        
    df['Granularidad'] = 'Diario'
    df['Indicador'] = 'UVR'
    
    df = df.loc[start:end, ['date', 'Granularidad', 'Indicador', 'Pesos colombianos por UVR']].reset_index().drop(['index'], axis=1)
    # bs.guardar_archivo(fuente_archivo, path+'/Data/Raw/UVR.xlsx')
    
    return df

In [40]:
extraer_UVR('01-01-1990', '01-01-2040')

            date Granularidad Indicador Pesos colombianos por UVR
0     01-01-2000       Diario       UVR                  103,3396
1     02-01-2000       Diario       UVR                  103,3556
2     03-01-2000       Diario       UVR                  103,3715
3     04-01-2000       Diario       UVR                  103,3875
4     05-01-2000       Diario       UVR                  103,4035
...          ...          ...       ...                       ...
8562  11-06-2023       Diario       UVR                  347,0641
8563  12-06-2023       Diario       UVR                  347,1511
8564  13-06-2023       Diario       UVR                  347,2381
8565  14-06-2023       Diario       UVR                  347,3252
8566  15-06-2023       Diario       UVR                  347,4122

[8567 rows x 4 columns]
